# Neural Network with Tensorflow

In the previous notebook, I created the algorithm to perform the classification of the MNIST data set. The MNIST data set has images of digits from 0-9. The task is assigned to each image of the digit its corresponding number. Here, I will use tensorflow to perform this task.

In [113]:
import tensorflow as tf
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [114]:
mnist = datasets.load_digits()
data = mnist.data
targets = mnist.target

In [115]:
def image_standarization(x):
    """
    Normalize a list of sample image data in the range of 0 to 1
    : x: List of image data.  The image shape is (32, 32, 3)
    : return: Numpy array of normalize data
    """
    x_demean = x - np.mean(x)
    adjusted_sd = np.maximum(np.std(x), 1.0/np.sqrt(np.prod(x.shape)))
    return  x_demean / adjusted_sd

In [116]:
#Standardizing data
data = np.apply_along_axis(image_standarization, 1, data)

In [117]:
#Separating train / test / validation
image, image_test, y, y_test = train_test_split(data, targets, test_size=0.2, train_size=0.8)
image_train, image_val, y_train, y_val = train_test_split(image, y,test_size = 0.25, train_size =0.75)

In [118]:
learning_rate = 0.0001

In [119]:
## Get batches
## Generating minibatches
## Taken from udacity helper function
def get_batch(batch_size, features, labels):
    for start in range(0, len(features), batch_size):
        end = min(start + batch_size, len(features))
        yield features[start:end], labels[start:end]

In [120]:
tf.reset_default_graph()
# Creating the graph
inputs_ = tf.placeholder(tf.float32, [None, 64])
labels_ = tf.placeholder(tf.int32, [None])
one_hot = tf.one_hot(indices =labels_, depth=10, axis=-1)

#Neural Network with hidden Rely and Softmax output layers
hidden_layer = tf.contrib.layers.fully_connected(inputs_, 20, activation_fn=tf.nn.relu)
logits = tf.layers.dense(hidden_layer, 10, activation=None)

#Calculating cost and defining optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot))
optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

#Calculating Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [121]:
epochs = 10000
batch_size = 128

In [125]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for e in range(epochs):
        for feature_batch, label_batch in get_batch(batch_size, image_train, y_train):
            loss, _ = sess.run([cost, optm], feed_dict={
                    inputs_: feature_batch,
                    labels_: label_batch
                })
            
        if e%1000 == 0:    
            print("Loss: {}".format(loss))
    
    #Each 4 iterations, check validation accuracy
        if e%1000 == 0:
            val_acc = sess.run(accuracy, feed_dict = {
                    inputs_: image_val,
                    labels_: y_val
                })
            print("Validation Accuracy: {}\n".format(val_acc))
    
    test_accuracy = sess.run(accuracy, feed_dict={
        inputs_: image_test,
        labels_: y_test
    })
    
    print("\nTest accuracy: {}\n".format(test_accuracy))
    


Loss: 3.4447460174560547
Validation Accuracy: 0.05833333358168602

Loss: 1.809862494468689
Validation Accuracy: 0.38055557012557983

Loss: 1.2941428422927856
Validation Accuracy: 0.6166666746139526

Loss: 0.9520531296730042
Validation Accuracy: 0.7388888597488403

Loss: 0.7320092916488647
Validation Accuracy: 0.8138889074325562

Loss: 0.5872597098350525
Validation Accuracy: 0.8472222089767456

Loss: 0.4920727610588074
Validation Accuracy: 0.8611111044883728

Loss: 0.4263288378715515
Validation Accuracy: 0.8722222447395325

Loss: 0.3789965510368347
Validation Accuracy: 0.8916666507720947

Loss: 0.3434933125972748
Validation Accuracy: 0.8999999761581421


Test accuracy: 0.8833333253860474

